<a href="https://colab.research.google.com/github/JungAh12/Everyone_TF2.0/blob/master/TF2_0_tensor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q tensorflow-gpu==2.0.0-rc1

     |████████████████████████████████| 380.5MB 43kB/s 
     |████████████████████████████████| 4.3MB 37.8MB/s 
     |████████████████████████████████| 501kB 57.1MB/s 


In [0]:
import tensorflow as tf

#tensor
텐서는 넘파이의 ndarray와 비슷하다.

`tf.Tensor`는 GPU, TPU같은 가속기 메모리에 상주할 수 있다.

In [4]:
print(tf.add(1,2))  #1+2
print(tf.add([1,2],[3,4]))  #[1+3, 2+4]
print(tf.square(5)) #5**2
print(tf.reduce_sum([1,2,3])) #1+2+3

tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor([4 6], shape=(2,), dtype=int32)
tf.Tensor(25, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)


각 텐서는 크기(shape)와 데이터 타입(dtype)을 가지고 있다.

In [5]:
x = tf.matmul([[1]],[[2,3]])
print(x)
print(x.shape)
print(x.dtype)

tf.Tensor([[2 3]], shape=(1, 2), dtype=int32)
(1, 2)
<dtype: 'int32'>


#Numpy의 배열과 tf.Tensor의 차이
1. 텐서는 가속기 메모리에서 사용가능하다.
2. 텐서는 불변성을 갖는다.

#Numpy와의 호환
tensorflow는 자동으로 numpy array를 tensor로 변환한다.

numpy는 자동으로 tensor를 numpy array로 변환한다.

`.numpy()`를 사용해서 넘파이 배열로 변환할 수 있다.

In [6]:
import numpy as np

ndarray = np.ones([3, 3]) # 1로 채워진 3x3 matrix

print("tf 연산은 자동으로 numpy array를 tensor로 변환한다.")
tensor = tf.multiply(ndarray, 42) #ndarray 각 요소에 42 곱함
print(tensor)

tf 연산은 자동으로 numpy array를 tensor로 변환한다.
tf.Tensor(
[[42. 42. 42.]
 [42. 42. 42.]
 [42. 42. 42.]], shape=(3, 3), dtype=float64)


In [7]:
print("numpy 연산은 자동으로 tensor를 numpy array로 변환한다.")
print(np.add(tensor, 1))    #tensor 각 요소에 1 더함

numpy 연산은 자동으로 tensor를 numpy array로 변환한다.
[[43. 43. 43.]
 [43. 43. 43.]
 [43. 43. 43.]]


In [8]:
print(".numpy()는 텐서를 넘파이배열로 변환한다.")
print(tensor.numpy())

.numpy()는 텐서를 넘파이배열로 변환한다.
[[42. 42. 42.]
 [42. 42. 42.]
 [42. 42. 42.]]


#GPU사용하기

In [9]:
x = tf.random.uniform([3,3]) #uniform distribution(균등분포)에 따라 랜덤으로 값을 추출하여 3x3 Tensor 생성

print("is gpu available"),
print(tf.test.is_gpu_available())

print("텐서가 GPU #0에 있는가")
print(x.device.endswith('GPU:0'))

## Colab에서는 하드웨어 세팅 변경해줘야함.

is gpu available
True
텐서가 GPU #0에 있는가
True


#장치 이름
`Tensor.device`는 텐서를 구성하는 호스트 장치의 풀네임을 제공한다.

텐서가 호스트의 N번째 GPU에 놓여지면 문자열은 GPU:N으로 나온다.

In [12]:
x.device

'/job:localhost/replica:0/task:0/device:GPU:0'

#명시적 장치 배치
`tf.device`를 사용해서 특정 장치에 텐서를 배치할 수 있다.

In [13]:
import time

def time_matmul(x):
    start = time.time()
    for loop in range(10):
        tf.matmul(x, x)
    
    result = time.time()-start
    print("10 loops: {:0.2f}ms".format(1000*result))

#CPU 실행
print("On CPU:")
with tf.device("CPU:0"):
    x = tf.random.uniform([1000,1000])  #1000x1000 tensor
    assert x.device.endswith("CPU:0")
    time_matmul(x)

#GPU 사용가능하면 GPU#0에서 강제 실행한다.
if tf.test.is_gpu_available():
    print("On GPU:")
    with tf.device("GPU:0"):
        x = tf.random.uniform([1000,1000])
        assert x.device.endswith("GPU:0")
        time_matmul(x)

On CPU:
10 loops: 320.58ms
On GPU:
10 loops: 1235.42ms


GPU가 더 오래 걸리는 이유는 아마 host(cpu)->device(gpu)로 복사할때 PCIe bus 속도가 느리기 때문일 것이다.

#Dataset
`tf.data.Dataset` API를 사용해서 파이프라인을 구축하자.

위 API는 모델을 training, evaluation loop를 제공할, 간단하고 재사용 가능한 모듈로부터 복잡한 입력 파이프라인을 구축하기 위해 사용된다.

#source dataset 생성하기
`Dataset.from_tensors, Dataset.from_tensor_slices`와 같은 팩토리 함수 중 하나를 사용하거나 파일로부터 읽어들이는 객체인 `TextLineDataset, TFRecordDataset`을 사용해서 소스 데이터셋을 생성할 수 있다.

In [0]:
ds_tensors = tf.data.Dataset.from_tensor_slices([1,2,3,4,5,6])

#CSV file 생성
import tempfile
_, filename = tempfile.mkstemp()

with open(filename, 'w') as f:
    f.write("""Line1
    Line2
    Line3
    """)

ds_file = tf.data.TextLineDataset(filename)

In [0]:
ds_tensors = ds_tensors.map(tf.square).shuffle(2).batch(2) #map(tf.square) = 각 요소 제곱, shuffle(2) = buffer에 2개 담아서 버퍼로부터 무작위 샘플링, batch(2) = 2개씩 배치로 묶음

ds_file = ds_file.batch(2)  #2개씩 배치로 묶음

In [28]:
print('ds_tensors elements:')
for x in ds_tensors:
    print(x)

print('\nds_file elements:')
for x in ds_file:
    print(x)

ds_tensors elements:
tf.Tensor([4 1], shape=(2,), dtype=int32)
tf.Tensor([ 9 25], shape=(2,), dtype=int32)
tf.Tensor([36 16], shape=(2,), dtype=int32)

ds_file elements:
tf.Tensor([b'Line1' b'    Line2'], shape=(2,), dtype=string)
tf.Tensor([b'    Line3' b'    '], shape=(2,), dtype=string)
